In [1]:
import numpy as np
import pandas as pd 
import math 

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.shape

(8925, 12)

In [4]:
test = pd.read_csv('test.csv')

In [5]:
train

,matchId,playerId,playType,bodyPart,x,y,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,outcome
0,m_91,p_103,جریان بازی,پای راست,13.47,-11.22,1,0,متوسط,70,9,گُل
1,m_17,p_16,جریان بازی,پای چپ,9.48,14.22,3,0,متوسط,55,4,مهار توسط دروازه بان
2,m_111,p_88,ضربه آزاد مستقیم,پای چپ,29.43,-1.25,6,2,کم,86,31,مهار توسط دروازه بان
3,m_142,p_87,جریان بازی,پای راست,26.93,1.00,4,1,متوسط,77,2,موقعیت از دست رفته
4,m_117,p_9,جریان بازی,پای راست,10.72,5.24,2,0,متوسط,76,46,گُل
...,...,...,...,...,...,...,...,...,...,...,...,...
8920,m_57,p_115,جریان بازی,سر,6.48,3.99,3,0,زیاد,69,50,موقعیت از دست رفته
8921,m_59,p_76,جریان بازی,پای راست,21.45,-8.73,4,1,متوسط,15,53,برخورد به دفاع
8922,m_55,p_150,جریان بازی,پای چپ,11.97,3.24,3,0,متوسط,84,34,موقعیت از دست رفته
8923,m_33,p_130,جریان بازی,پای راست,6.48,-6.98,1,0,زیاد,4,39,موقعیت از دست رفته


In [6]:
test

,minute,second,x,y,playType,bodyPart,interveningOpponents,interveningTeammates,interferenceOnShooter
0,76,7,6.98,-5.49,جریان بازی,سر,1,0,کم
1,81,18,27.43,1.00,جریان بازی,پای چپ,2,1,کم
2,86,50,13.47,-1.00,جریان بازی,پای چپ,4,0,متوسط
3,69,39,9.23,-9.23,جریان بازی,پای راست,1,0,زیاد
4,53,15,10.72,-8.48,جریان بازی,پای راست,2,1,کم
...,...,...,...,...,...,...,...,...,...
1995,86,21,7.23,12.47,جریان بازی,پای راست,1,0,کم
1996,11,19,11.22,8.48,جریان بازی,پای چپ,2,0,کم
1997,8,33,11.47,-1.25,جریان بازی,پای راست,2,0,متوسط
1998,53,42,18.45,-14.71,جریان بازی,پای راست,4,0,متوسط


In [7]:
def labeling (i):
    if i=="گُل" or i=="گُل به خودی":
        return 1
    else:
      return 0

In [8]:
train["label"] = train["outcome"].apply(labeling)

In [9]:
train = train.drop('outcome', axis=1)

In [10]:
train = train.drop(['playerId','matchId'], axis=1)

In [11]:
def bodyPart (i):
    if i=='پای چپ' or i=="پای راست":
        return 'پا'
    else:
      return i

In [12]:
train['bodyPart']=train['bodyPart'].apply(bodyPart)
test['bodyPart']=test['bodyPart'].apply(bodyPart)

In [13]:
x_2= pow(train["x"],2)
y_2= pow(train["y"],2)
train['distance']= x_2+y_2
train['distance']= train['distance'].apply( math.sqrt )

In [14]:
x_2test= pow(test["x"],2)
y_2test= pow(test["y"],2)
test['distance']= x_2test+y_2test
test['distance']= test['distance'].apply( math.sqrt )

In [15]:
j= pow(7.32/2,2)

In [16]:
train["angle"] = (7.32*train["x"])/(x_2 + y_2 -j )

In [17]:
test["angle"] = (7.32*test["x"])/(x_2test + y_2test -j )

In [18]:
def angle(x):
  if x>=0:
    return np.rad2deg(np.arctan(x))
  else:
    return np.rad2deg(np.arctan(x) + np.pi)

In [19]:
train['angle'] = train["angle"].apply(angle)
test['angle'] = test["angle"].apply(angle)

In [20]:
train = train.drop(['x','y'], axis=1)
test = test.drop(['x','y'], axis=1)

In [21]:
def fill_missing(y):
  if y==0:
    return "کم"
  elif y==1:
    return "متوسط"
  else:
    return "زیاد"

In [22]:
y= train[train['interferenceOnShooter'].isna()]['interveningOpponents'].apply(fill_missing)
train['interferenceOnShooter'].fillna(y, inplace= True)

In [23]:
y_test= test[test['interferenceOnShooter'].isna()]['interveningOpponents'].apply(fill_missing)
test['interferenceOnShooter'].fillna(y_test, inplace= True)

In [24]:
interferenceOnShooter = {"کم": 0,
           'متوسط' : 1,
           'زیاد' : 2}

train['interferenceOnShooter'] = train['interferenceOnShooter'].map(interferenceOnShooter)
test['interferenceOnShooter'] = test['interferenceOnShooter'].map(interferenceOnShooter)

In [25]:
v = pd.get_dummies(train['bodyPart'],prefix='bodyPart')
w = pd.get_dummies(train['playType'],prefix='playType')
z = pd.get_dummies(train['interferenceOnShooter'],prefix='interferenceOnShooter')
train = pd.concat([train, v,w,z], axis=1)

train.head()

,playType,bodyPart,interveningOpponents,interveningTeammates,interferenceOnShooter,minute,second,label,distance,angle,bodyPart_سایر,bodyPart_سر,bodyPart_پا,playType_جریان بازی,playType_ضربه آزاد مستقیم,playType_مستقیم از کرنر,playType_پنالتی,interferenceOnShooter_0,interferenceOnShooter_1,interferenceOnShooter_2
0,جریان بازی,پا,1,0,1,70,9,1,17.530810,18.544088,0,0,1,1,0,0,0,0,1,0
1,جریان بازی,پا,3,0,1,55,4,0,17.090313,13.982592,0,0,1,1,0,0,0,0,1,0
2,ضربه آزاد مستقیم,پا,6,2,0,86,31,0,29.456534,14.153255,0,0,1,0,1,0,0,1,0,0
3,جریان بازی,پا,4,1,1,77,2,0,26.948560,15.458384,0,0,1,1,0,0,0,0,1,0
4,جریان بازی,پا,2,0,1,76,46,1,11.932141,31.315918,0,0,1,1,0,0,0,0,1,0


In [26]:
vt = pd.get_dummies(test['bodyPart'],prefix='bodyPart')
wt = pd.get_dummies(test['playType'],prefix='playType')
zt = pd.get_dummies(test['interferenceOnShooter'],prefix='interferenceOnShooter')
test = pd.concat([test, vt,wt,zt], axis=1)

test.head()

,minute,second,playType,bodyPart,interveningOpponents,interveningTeammates,interferenceOnShooter,distance,angle,bodyPart_سایر,bodyPart_سر,bodyPart_پا,playType_جریان بازی,playType_ضربه آزاد مستقیم,playType_مستقیم از کرنر,playType_پنالتی,interferenceOnShooter_0,interferenceOnShooter_1,interferenceOnShooter_2
0,76,7,جریان بازی,سر,1,0,0,8.880343,37.971084,0,1,0,1,0,0,0,1,0,0
1,81,18,جریان بازی,پا,2,1,0,27.448222,15.180640,0,0,1,1,0,0,0,1,0,0
2,86,50,جریان بازی,پا,4,0,1,13.507069,30.254071,0,0,1,1,0,0,0,0,1,0
3,69,39,جریان بازی,پا,1,0,2,13.053191,23.285510,0,0,1,1,0,0,0,0,0,1
4,53,15,جریان بازی,پا,2,1,0,13.668533,24.344524,0,0,1,1,0,0,0,1,0,0


In [27]:
train = train.drop(['bodyPart','playType','interferenceOnShooter'], axis=1)
test = test.drop(['bodyPart','playType','interferenceOnShooter'], axis=1)

In [28]:
train

,interveningOpponents,interveningTeammates,minute,second,label,distance,angle,bodyPart_سایر,bodyPart_سر,bodyPart_پا,playType_جریان بازی,playType_ضربه آزاد مستقیم,playType_مستقیم از کرنر,playType_پنالتی,interferenceOnShooter_0,interferenceOnShooter_1,interferenceOnShooter_2
0,1,0,70,9,1,17.530810,18.544088,0,0,1,1,0,0,0,0,1,0
1,3,0,55,4,0,17.090313,13.982592,0,0,1,1,0,0,0,0,1,0
2,6,2,86,31,0,29.456534,14.153255,0,0,1,0,1,0,0,1,0,0
3,4,1,77,2,0,26.948560,15.458384,0,0,1,1,0,0,0,0,1,0
4,2,0,76,46,1,11.932141,31.315918,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8920,3,0,69,50,0,7.609895,46.818116,0,1,0,1,0,0,0,0,0,1
8921,4,1,15,53,0,23.158484,16.713121,0,0,1,1,0,0,0,0,1,0
8922,3,0,84,34,0,12.400746,31.970470,0,0,1,1,0,0,0,0,1,0
8923,1,0,4,39,0,9.524222,31.529506,0,0,1,1,0,0,0,0,0,1


In [29]:
test

,minute,second,interveningOpponents,interveningTeammates,distance,angle,bodyPart_سایر,bodyPart_سر,bodyPart_پا,playType_جریان بازی,playType_ضربه آزاد مستقیم,playType_مستقیم از کرنر,playType_پنالتی,interferenceOnShooter_0,interferenceOnShooter_1,interferenceOnShooter_2
0,76,7,1,0,8.880343,37.971084,0,1,0,1,0,0,0,1,0,0
1,81,18,2,1,27.448222,15.180640,0,0,1,1,0,0,0,1,0,0
2,86,50,4,0,13.507069,30.254071,0,0,1,1,0,0,0,0,1,0
3,69,39,1,0,13.053191,23.285510,0,0,1,1,0,0,0,0,0,1
4,53,15,2,1,13.668533,24.344524,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,86,21,1,0,14.414361,15.230798,0,0,1,1,0,0,0,1,0,0
1996,11,19,2,0,14.064096,24.007469,0,0,1,1,0,0,0,1,0,0
1997,8,33,2,0,11.537911,35.040470,0,0,1,1,0,0,0,0,1,0
1998,53,42,4,0,23.596326,13.957446,0,0,1,1,0,0,0,0,1,0


In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train[['interveningOpponents']] = scaler.fit_transform(train[['interveningOpponents']])
train[['interveningTeammates']] = scaler.fit_transform(train[['interveningTeammates']])
train[['minute']] = scaler.fit_transform(train[['minute']])
train[['second']] = scaler.fit_transform(train[['second']])
train[['distance']] = scaler.fit_transform(train[['distance']])
train[['angle']] = scaler.fit_transform(train[['angle']])


test[['interveningOpponents']] = scaler.fit_transform(test[['interveningOpponents']])
test[['interveningTeammates']] = scaler.fit_transform(test[['interveningTeammates']])
test[['minute']] = scaler.fit_transform(test[['minute']])
test[['second']] = scaler.fit_transform(test[['second']])
test[['distance']] = scaler.fit_transform(test[['distance']])
test[['angle']] = scaler.fit_transform(test[['angle']])

In [31]:
y=  train[['label']]

In [32]:
x = train.copy()
x.drop(columns= ['label'], inplace= True)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(x,y)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
y_pred = LR.predict(x)

In [35]:
from sklearn.metrics import jaccard_score
jaccard_score(y, y_pred,pos_label=1)

0.1761786600496278

In [36]:
y_pred_prob = LR.predict_proba(x)


In [37]:
from sklearn.metrics import log_loss
log_loss(y, y_pred_prob)

0.304431326666674

In [38]:
y_test_pred = LR.predict_proba(test)
y_test_pred[:,1]

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


array([0.11543941, 0.03639235, 0.07645317, ..., 0.25491395, 0.05261492,
       0.08228332])

In [39]:
y_test_pred

array([[0.88456059, 0.11543941],
       [0.96360765, 0.03639235],
       [0.92354683, 0.07645317],
       ...,
       [0.74508605, 0.25491395],
       [0.94738508, 0.05261492],
       [0.91771668, 0.08228332]])

In [40]:
csv_new = pd.DataFrame(y_test_pred[:,1], columns=['prediction'])
csv_new

,prediction
0,0.115439
1,0.036392
2,0.076453
3,0.067415
4,0.133143
...,...
1995,0.073642
1996,0.234013
1997,0.254914
1998,0.052615


In [41]:
csv_new.to_csv('output.csv',index=False)